In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [2]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

ModuleNotFoundError: No module named 'kobert'

In [3]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

: 

: 

In [4]:
##GPU 사용 시
device = torch.device("cuda:0")

In [5]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [6]:
dataset_train = nlp.data.TSVDataset("./ratings_train.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("./ratings_test.txt", field_indices=[1,2], num_discard_samples=1)

In [7]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model


In [8]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [9]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [11]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=0)

In [12]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [13]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [14]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [15]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [16]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [17]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [18]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [19]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

torch.save(model.state_dict(), "pretrained_01.pt")
torch.save(model, "pretrained_02.pt")


ipykernel_launcher:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7433879971504211 train acc 0.4375
epoch 1 batch id 201 loss 0.528231680393219 train acc 0.5464085820895522
epoch 1 batch id 401 loss 0.5216503143310547 train acc 0.6558213840399002
epoch 1 batch id 601 loss 0.40676867961883545 train acc 0.710404534109817
epoch 1 batch id 801 loss 0.46039101481437683 train acc 0.7427434456928839
epoch 1 batch id 1001 loss 0.3263438642024994 train acc 0.7630962787212787
epoch 1 batch id 1201 loss 0.38278070092201233 train acc 0.7777763322231473
epoch 1 batch id 1401 loss 0.4182613492012024 train acc 0.7885662027123483
epoch 1 batch id 1601 loss 0.337914377450943 train acc 0.7976069643972518
epoch 1 batch id 1801 loss 0.2855183482170105 train acc 0.8049521099389229
epoch 1 batch id 2001 loss 0.32992810010910034 train acc 0.8114926911544228
epoch 1 batch id 2201 loss 0.3138485252857208 train acc 0.8169936960472512
epoch 1 train acc 0.8206924594709898
ipykernel_launcher:22: TqdmDeprecationWarning: This function will be removed in t

  0%|          | 0/782 [00:00<?, ?it/s]

epoch 1 test acc 0.8796355498721228


...

epoch 4 test acc 0.8960398017902813


  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.45540353655815125 train acc 0.875
epoch 5 batch id 201 loss 0.07725607603788376 train acc 0.9742692786069652
epoch 5 batch id 401 loss 0.09456852078437805 train acc 0.975218204488778
epoch 5 batch id 601 loss 0.19975478947162628 train acc 0.9751975873544093
epoch 5 batch id 801 loss 0.12755051255226135 train acc 0.9756554307116105
epoch 5 batch id 1001 loss 0.015599188394844532 train acc 0.9762112887112887
epoch 5 batch id 1201 loss 0.01966099441051483 train acc 0.9766340549542049
epoch 5 batch id 1401 loss 0.020209617912769318 train acc 0.9768692005710207
epoch 5 batch id 1601 loss 0.008560390211641788 train acc 0.9772212679575265
epoch 5 batch id 1801 loss 0.005831775721162558 train acc 0.9775992504164354
epoch 5 batch id 2001 loss 0.04955450817942619 train acc 0.9779641429285357
epoch 5 batch id 2201 loss 0.02135828137397766 train acc 0.978120740572467
epoch 5 train acc 0.978448965443686


  0%|          | 0/782 [00:00<?, ?it/s]

epoch 5 test acc 0.895400415601023
